# Prepping

In [64]:
import scipy.linalg as la
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

In [3]:
M = 100. * (np.random.rand(10, 10) - 0.5)
M = 0.5 * (M - M.T);
R = expm(M);

In [4]:
n = 500
Xt = 10. * np.random.rand(n, 2);
X = np.append(Xt, 0.5 * np.random.randn(n, 8), 1);

# Rotate data randomly.
X = np.dot(X, R.T);
k = 10
epsilon = 1.8
tol=1e-3

# LEE General

In [223]:
    #Add value errors for: negative k/epsilon, too large k
distances = -2 * X @ X.T + np.sum(X ** 2, axis=1) + np.sum(X ** 2, axis=1)[:, np.newaxis]

distances[distances < 0] = 0

distances = distances ** .5
indices = np.argsort(distances, 0)

10

In [6]:
#knn rule:
knnindices = indices[:, 0:k]

In [225]:
#epsilon rule:
d = X.shape[1]
mask = (distances<epsilon) & (distances>0)
epsilonmask = np.where(mask, True, False)
#epsilonmask_matrix = np.where(mask, [np.tile([True], d)], [np.tile([False], d)])

ValueError: operands could not be broadcast together with shapes (500,500) (1,10) (1,10) 

# Epsilonball, 1st markdown = for 1 index

neighIndex = indices[11][np.where(epsilonmask[11])]
neighs = X[neighIndex]
neighs = neighs - X[11]
C = neighs@neighs.T + tol
loc_W = la.solve(C, np.ones(C.shape[0]).T, sym_pos=True)
loc_W = loc_W/loc_W.sum()
W = np.zeros(len(X))
W[epsilonmask[11]] = loc_W

CHECK FOR REACHABILITY!!
scipy.sparse.csgraph.connected_components to check if the graph is connected ? 

In [8]:
#This is just me trying out stuff
neighIndex = indices[11][np.where(epsilonmask[11])]
neighIndex.shape

Plan: Matrix
    High level:
    Per dataset 
    for each (500) datapoint
        All relative positions (relPos) (500x500xD)
    relPos*mask = neighbours, all others are nulled; nulling needs to be done differently
    neighbours @ neighbours.T (neigh[x]*neigh[x].T) ->Covariance, after this all the same

In [202]:
relPos = X - X[:, np.newaxis]

In [259]:
neighbours = np.einsum('ijk,ij->ijk', relPos, epsilonmask)

# Below is only more or less useful me trying out stuff
Interesting from an efficiency perspective would maybe be the reshape instead of the einsum used, as this is supposedly basically a free action

In [292]:
#This is just me trying out stuff
A = [1,6,2,4,8]
B = [5,3,7,4,2]
C = [4,2,6,3,6]
D = [6,2,87,3,4]
nu = [0,0,0,0,0]
F = np.array([A, B,C,D])
Ft = np.array([A, B,nu,C,D])
G = np.array([B,D, A,C])
b= F@F.T+tol
a=Ft@Ft.T+tol

In [293]:
#This is just me trying out stuff
loc_W = la.solve(b, np.ones(b.shape[0]).T, sym_pos=True)
loc_A = la.solve(a, np.ones(a.shape[0]).T, sym_pos=True)

In [294]:
#This is just me trying out stuff
loc_W

array([-0.00503058,  0.01504871,  0.01160491, -0.00191064])

In [295]:
#This is just me trying out stuff
loc_A

array([ 1.48788109e-18, -2.60916654e-18,  1.00000000e+03,  3.74337540e-18,
        9.49518749e-20])

In [285]:
#Pretty much working on an single index level (of course hard to test but looks fine as far as I can tell so far)
neighIndex = indices[11][np.where(epsilonmask[11])]
neighs = X[neighIndex]
neighs = neighs - X[11]
C = neighs@neighs.T + tol
loc_W = la.solve(C, np.ones(C.shape[0]).T, sym_pos=True)
loc_W = loc_W/loc_W.sum()
W = np.zeros(len(X))
W[epsilonmask[11]] = loc_W

In [41]:
#This is just me trying out stuff
t = np.array([[1, 1, 1, 1],[2, 3, 4, 1], [2, 2, 2, 2]])

In [42]:
#This is just me trying out stuff
(-2*t@t.T +np.sum(t, axis=1) + np.sum(t, axis=1)[:, np.newaxis])

array([[  0,  -6,  -4],
       [ -6, -40, -22],
       [ -4, -22, -16]])

In [52]:
#This is just me trying out stuff
t.shape

(3, 4)

In [130]:
#This is just me trying out stuff
zeroElement = np.zeros(t.shape[1])
w = np.array([[len(t) * t.shape[1]], [len(t) * t.shape[1]]])


ValueError: operands could not be broadcast together with shape (4,) (2,)

In [125]:
#This is just me trying out stuff

mat = w
count1 = 0
for node in t:
    count2 = 0
    for gNode in t:
        w[count1][count2] = (gNode-node)
        count2 = count2+1
        print(count1)
    count1 = count1+1

ValueError: setting an array element with a sequence.

In [162]:
#This is just me trying out stuff
z = np.tile([[zeroElement]], (2, 1))

In [166]:
#This is just me trying out stuff
z = np.zeros((2,2,4))

In [167]:
#This is just me trying out stuff
z

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

(500, 500, 10)

In [228]:
#This is just me trying out stuff
A = np.array([[2,5,3],[4,5,7], [8,4,6]])
B = np.array([4,5,6])
B*A

array([[ 8, 25, 18],
       [16, 25, 42],
       [32, 20, 36]])

In [230]:
#This is just me trying out stuff
np.einsum('ijk,lk->ijl', relPos, epsilonmask)

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (500,500,10)->(500,500,newaxis,10) (500,500)->(500,500) 

In [238]:
#This is just me trying out stuff
res_shape = epsilonmask.shape[:1] + relPos.shape[1:]
res = (epsilonmask * relPos.reshape(relPos.shape[0], -1)).reshape(res_shape)

(500, 500, 10)


ValueError: operands could not be broadcast together with shapes (500,500) (500,5000) 

In [ ]:
#This is just me trying out stuff
res_shape = S_pinv.shape[:1] + images.shape[1:]  # (3, 5, 6) 
res = (S_pinv @ images.reshape(images.shape[0], -1)).reshape(res_shape)
print(res.shape)  # (3, 5, 6)

In [237]:
#This is just me trying out stuff
S_pinv = np.random.rand(3, 4)
images = np.random.rand(4, 5, 6)

# error: 
# (S_pinv @ images).shape 
res_shape = S_pinv.shape[:1] + images.shape[1:]  # (3, 5, 6) 
res = (S_pinv @ images.reshape(images.shape[0], -1)).reshape(res_shape)
print(res.shape)  # (3, 5, 6)

(3, 5, 6)


In [239]:
#This is just me trying out stuff
S_pinv

array([[0.95076507, 0.96364309, 0.13587741, 0.15226442],
       [0.06850193, 0.57791577, 0.26783793, 0.92712151],
       [0.88292935, 0.83496219, 0.8810834 , 0.38592518]])

In [241]:
#This is just me trying out stuff
images[0]

array([[0.92875414, 0.33515343, 0.64413908, 0.24481038, 0.56601181,
        0.48455075],
       [0.7538109 , 0.75834894, 0.52430422, 0.54724836, 0.03303771,
        0.55047272],
       [0.45909714, 0.11757157, 0.48042486, 0.91362234, 0.27206296,
        0.33705151],
       [0.33538448, 0.52873799, 0.48280649, 0.41800567, 0.92246233,
        0.56661495],
       [0.4833479 , 0.90881075, 0.804039  , 0.01090026, 0.40804685,
        0.42072832]])

In [243]:
#This is just me trying out stuff
images.reshape(images.shape[0], -1)[0]

array([0.92875414, 0.33515343, 0.64413908, 0.24481038, 0.56601181,
       0.48455075, 0.7538109 , 0.75834894, 0.52430422, 0.54724836,
       0.03303771, 0.55047272, 0.45909714, 0.11757157, 0.48042486,
       0.91362234, 0.27206296, 0.33705151, 0.33538448, 0.52873799,
       0.48280649, 0.41800567, 0.92246233, 0.56661495, 0.4833479 ,
       0.90881075, 0.804039  , 0.01090026, 0.40804685, 0.42072832])

In [245]:
#This is just me trying out stuff
relPos.reshape(relPos.shape[0], -1)[0]

array([0.        , 0.        , 0.        , ..., 1.27110611, 2.25460945,
       2.17659331])

In [247]:
#This is just me trying out stuff
relPos[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [258]:
#This is just me trying out stuff
np.einsum('ijk,ij->ijk', relPos, epsilonmask)[2][35]

array([ 0.62643731, -0.23051   , -0.29934212,  0.22977954, -0.01387712,
       -0.05670286,  0.60643149,  0.75558609,  0.39334547, -0.46520408])

In [257]:
#This is just me trying out stuff
epsilonmask[2][35]

True